In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/bike-sharing-demand/sampleSubmission.csv
/kaggle/input/bike-sharing-demand/train.csv
/kaggle/input/bike-sharing-demand/test.csv


In [2]:
# creating file paths
bike_sharing_file_train = '../input/bike-sharing-demand/train.csv'
bike_sharing_file_test = '../input/bike-sharing-demand/test.csv'
bike_data_train = pd.read_csv(bike_sharing_file_train)
bike_data_test = pd.read_csv(bike_sharing_file_test)

# dropping null values
bike_data_train = bike_data_train.dropna(axis = 0)
bike_data_test = bike_data_test.dropna(axis = 0)

# NEED TO CHANGE 'datetime' VALUES TO 'timeofday' 
bike_data_train['datetime'] = pd.to_datetime(bike_data_train['datetime'])
bike_data_train['timeofday'] = bike_data_train['datetime'].dt.hour

bike_data_test['datetime'] = pd.to_datetime(bike_data_test['datetime'])
bike_data_test['timeofday'] = bike_data_test['datetime'].dt.hour

# setting prediction target
y = bike_data_train['count']

# choosing features for ML model
bike_data_features = ['timeofday', 'season', 'holiday', 'weather', 'temp', 'windspeed', 'humidity', 'workingday']

X = bike_data_train[bike_data_features]
test_X = bike_data_test[bike_data_features]

In [3]:
# testing first model in-sample
# setting model
bike_model_DT = DecisionTreeRegressor(random_state = 1)

# fit model
bike_model_DT.fit(X, y)

predicted_count = bike_model_DT.predict(X)
print("In-Sample MAE Model 1:", mean_absolute_error(y, predicted_count))

In-Sample MAE Model 1: 0.5512278767836364


In [4]:
# testing validity using MAE
# splitting train data to test
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

# setting model
bike_model_DT = DecisionTreeRegressor(random_state = 1)

# fit model
bike_model_DT.fit(train_X, train_y)

# calculating MAE
val_predictions = bike_model_DT.predict(val_X)
print("Validition Data MAE Model 1:", mean_absolute_error(val_y, val_predictions))

Validition Data MAE Model 1: 60.92940240019594


In [5]:
# trying new model
bike_model_RF = RandomForestRegressor(random_state = 1)

# fitting new model
bike_model_RF.fit(train_X, train_y)

# calculating MAE
val_predictions_2 = bike_model_RF.predict(val_X)
print("Validition Data MAE Model 2:", mean_absolute_error(val_y, val_predictions_2))

Validition Data MAE Model 2: 47.28516767257968


In [6]:
# preparing to submit model
# fitting second model on full set of training data
bike_model_final = RandomForestRegressor()
bike_model_final.fit(X, y)

# making predictions for submission
test_predictions = bike_model_final.predict(test_X)

# output to csv
output = pd.DataFrame({'datetime': bike_data_test.datetime,
                       'count': test_predictions})
output.to_csv('submission_bike.csv', index=False)